In [55]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.discriminant_analysis import StandardScaler
from sklearn.metrics import accuracy_score

# Load data function

In [56]:
def load_data(filename):
    data = pd.read_csv(filename)
    X = data.drop('Revenue', axis=1) 
    y = data['Revenue']  

    return X, y

# Encoded features function

In [57]:
def encode_features(X):
    label_encoder = LabelEncoder()

    # Label encoding for 'Month'
    X['Month'] = label_encoder.fit_transform(X['Month'])

    # Label encoding for 'VisitorType'
    X['VisitorType'] = label_encoder.fit_transform(X['VisitorType'])

    return X

# Relu function

In [58]:
def relu(z):
    return np.maximum(0, z)

# Sigmoid function

In [59]:
def sigmoid(z):
    z = np.clip( z, -500, 500 )           # protect against overflow
    g = 1.0/(1.0+np.exp(-z))

    return g

# My dense vector function

In [60]:
def my_dense_v(A_in, W, b, g):
    Z = np.matmul(A_in, W) + b
    A_out = g(Z)
    return(A_out)

# My sequential function

In [61]:
def my_sequential_v(X, W1, b1, W2, b2, W3, b3):
    A1 = my_dense_v(X,  W1, b1, relu)
    A2 = my_dense_v(A1, W2, b2, relu)
    A3 = my_dense_v(A2, W3, b3, sigmoid)
    return(A3)

# Main 

In [62]:
# Load and preprocess data
X, y = load_data("dataset.csv")
X_encoded = encode_features(X)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and train TensorFlow model
model = Sequential([
    tf.keras.Input(shape=(17,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
], name="my_model")

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(0.001))
model.fit(X_train_scaled, y_train, epochs=20)

# Extract weights after training
[layer1, layer2, layer3] = model.layers
W1_tmp, b1_tmp = layer1.get_weights()
W2_tmp, b2_tmp = layer2.get_weights()
W3_tmp, b3_tmp = layer3.get_weights()

# Use custom model for predictions on test data
custom_predictions = my_sequential_v(X_test_scaled, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp)
custom_predicted_classes = (custom_predictions > 0.5).astype(int)
custom_accuracy = accuracy_score(y_test, custom_predicted_classes)
print("Custom Model Accuracy:", custom_accuracy)

# TensorFlow model predictions on test data
tf_predictions = model.predict(X_test_scaled)
tf_predicted_classes = (tf_predictions > 0.5).astype(int)
tf_accuracy = accuracy_score(y_test, tf_predicted_classes)
print("TensorFlow Model Accuracy:", tf_accuracy)


Epoch 1/20
309/309 [==============================] - 1s 1ms/step - loss: 0.3181
Epoch 2/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2587
Epoch 3/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2483
Epoch 4/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2434
Epoch 5/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2393
Epoch 6/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2362
Epoch 7/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2323
Epoch 8/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2298
Epoch 9/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2270
Epoch 10/20
309/309 [==============================] - 0s 2ms/step - loss: 0.2250
Epoch 11/20
309/309 [==============================] - 0s 1ms/step - loss: 0.2237
Epoch 12/20
309/309 [==============================] - 1s 2ms/step - loss: 0.2213
Epoch 13/20
309/309 [====

Prediction = my_sequential_v(X, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
Prediction.shape